# Recurrent Network

This notebook shows how to create a recurrent network

### Import all the needed modules

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error

### Generate random values and calculate the label using the toy problem

The model shold be able to learn how to calculate the standard deviation of a sequence

To define data with different lengths, we set the unused parts of the data as -1.

In [2]:
N_SAMPLES = 100000
N_TEST = 1000
MAX_TIMESTEPS = 10
MASK_VALUE = -1

train_X = np.random.uniform(size=(N_SAMPLES, MAX_TIMESTEPS, 1))
train_L = np.random.randint(2, MAX_TIMESTEPS, N_SAMPLES)

test_X = np.random.uniform(size=(N_TEST, MAX_TIMESTEPS, 1))
test_L = np.random.randint(2, MAX_TIMESTEPS, N_TEST)

In [3]:
for i in range(N_SAMPLES):
    train_X[i, train_L[i]:] = MASK_VALUE

In [4]:
for i in range(N_TEST):
    test_X[i, test_L[i]:] = MASK_VALUE

In [5]:
train_y = np.ma.masked_array(train_X, train_X==MASK_VALUE).std(axis=1).data
test_y = np.ma.masked_array(test_X, test_X==MASK_VALUE).std(axis=1).data

### Define the connections of the nodes in the network and verify the model

The ```Masking``` layer removes the numbers whose values are equal to -1. This allows us to tell the network that the data lengths are different.

The ```return_sequence``` flag defines if the LSTM should return only the last or the full sequence

In [6]:
input_ = tf.keras.Input(shape=(None, 1))
masked = tf.keras.layers.Masking(MASK_VALUE)(input_)
lstm1 = tf.keras.layers.LSTM(32, return_sequences=True)(masked)
lstm2 = tf.keras.layers.LSTM(32)(lstm1)
output = tf.keras.layers.Dense(1)(lstm2)

model = tf.keras.Model(inputs=input_, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
masking (Masking)            (None, None, 1)           0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 32)          4352      
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 12,705
Trainable params: 12,705
Non-trainable params: 0
_________________________________________________________________


### Compile the model by providing the optimization algorithm and the loss function

In [7]:
model.compile('adam', 'mse')

### Train the model for 3 epochs

In [8]:
hist = model.fit(train_X, train_y, epochs=3)

Train on 100000 samples
Epoch 1/3
100000/100000 [==============================] - 104s 1ms/sample - loss: 0.0035
Epoch 2/3
100000/100000 [==============================] - 104s 1ms/sample - loss: 2.8249e-04
Epoch 3/3
100000/100000 [==============================] - 90s 902us/sample - loss: 1.4435e-04


### View the predictions on the test set

In [9]:
prediction = model.predict(test_X)

In [10]:
prediction[:5]

array([[0.22712992],
       [0.2701319 ],
       [0.19487977],
       [0.29359826],
       [0.37256256]], dtype=float32)

In [11]:
test_y[:5]

array([[0.22016771],
       [0.26719338],
       [0.18735555],
       [0.28748393],
       [0.3667492 ]])

In [12]:
mean_absolute_error(test_y, prediction)

0.00863914644851209